In [1]:
import json
import nltk
from nltk.tokenize import RegexpTokenizer
import string
import math
import time
import pandas as pd
import pyspark
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.classification import SVMWithSGD, SVMModel
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import random
import cPickle as pickle

from sklearn.metrics import coverage_error
from scipy.sparse import dok_matrix
import sklearn.feature_extraction
import sklearn.metrics


In [2]:
IS_SAMPLE = False

In [3]:
MIN_DOCUMENTS = 5
TOP_N_FEATURES = 10000

SVM_ITERATIONS = 1000
SVM_REG = 0.1

BM25_K = 1.5  # controls power of tf component
BM25_b = 0.75  # controls the BM25 length normalization

RANDOM_SEED = 10000
random.seed(RANDOM_SEED)

### Training functions

In [4]:
def merge_postings(postings_list1, postings_list2):
    # key could be either a doc id or a term
    for key in postings_list2:
        if postings_list1.get(key):
            postings_list1[key] += postings_list2[key]
        else:
            postings_list1[key] = postings_list2[key]
    return postings_list1

def get_term_dictionary(terms):
    """
    Maps string terms to indexes in an array
    """
    term_dictionary = {}
    term_array = [None] * len(terms)
    def put(key):
        hashvalue = hashfunction(key, len(term_array))
        if term_array[hashvalue] == None:
            term_array[hashvalue] = key
            return hashvalue
        else:
            nextslot = rehash(hashvalue, len(term_array))
            while term_array[nextslot] != None:
                nextslot = rehash(nextslot, len(term_array))
            if term_array[nextslot] == None:
                term_array[nextslot] = key
                return nextslot
    def hashfunction(key, size):
        return hash(key) % size
    def rehash(oldhash, size):
        return (oldhash + 1) % size
    i = 0
    for term in terms:
        corresponding_index = put(term)
        term_dictionary[term] = corresponding_index
        i+=1
        if i%10000 == 0: print "finished " + str(i)
    return term_dictionary

def jsonKV2str(x):
    """
    Change string keys to int
    """
    if isinstance(x, dict):
            #return {doc_id:{int(term_id):x[doc_id][term_id] for term_id in x[doc_id]} for doc_id in x }
        
            return {int(k):(int(v) if isinstance(v, unicode) else v) for k,v in x.items()}
    return x

def get_json(json_postings):
    return json.loads(json_postings)

def get_json_convert_num(json_postings):
    return json.loads(json_postings, object_hook=jsonKV2str)

def get_doc_index(term, postings_list, term_dictionary):
    #return [(doc_id, {term: postings_list[doc_id]}) for doc_id in postings_list]
    return [(doc_id, {term_dictionary[term]: postings_list[doc_id]}) for doc_id in postings_list]

def get_classes(ipc_classification):
    sections = []
    classes = []
    subclasses = []
    for classification in ipc_classification:
        # we do the check because some documents have repetitions
        section_name = classification['section']
        class_name = classification['section'] + "-" + classification['class']
        subclass_name = classification['section'] + "-" + classification['class'] + "-" + classification['subclass']
        if section_name not in sections:
            sections.append(section_name)
        if class_name not in classes:
            classes.append(class_name)
        if subclass_name not in subclasses:
            subclasses.append(subclass_name)
    return {"sections": sections, "classes": classes, "subclasses": subclasses}


def compare_classifications(x,y):
    len_comp = cmp(len(x), len(y))
    if len_comp == 0:
        return cmp(x,y)
    return len_comp


def create_doc_index(term_index, term_dictionary):
    return term_index \
        .flatMap(lambda (term, postings_list): get_doc_index(term, postings_list, term_dictionary)) \
        .reduceByKey(lambda x, y: merge_postings(x, y))


def get_rf_stats(postings, classification):
    a_plus_c = set(postings.keys())
    a_plus_b = set(classifications_index[classification])
    # first intersection is to get (a), second difference is to get (c) (checkout tf-rf paper for reference)
    a = a_plus_c.intersection(a_plus_b)
    c = a_plus_c.difference(a_plus_b)
    size_a = len(a)
    size_c = len(c)
    return size_a, size_c


def get_rf_postings(classification):
    def get_rf_postings_internal(postings):
        size_a, size_c = get_rf_stats(postings, classification)
        return {docId: calculate_rf(size_a, size_c)
                for docId, tf in postings.items()}
    return get_rf_postings_internal


def get_tf_rf_postings(classification):
    def get_tf_rf_postings_internal(postings):
        size_a, size_c = get_rf_stats(postings, classification)
        return {docId: calculate_tf_rf(tf, size_a, size_c)
                for docId, tf in postings.items()}
    return get_tf_rf_postings_internal

def get_error(svm, test_vectors):
    labelsAndPreds = test_vectors.map(lambda p: (p.label, svm.predict(p.features)))
    trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(test_vectors.count())
    return trainErr


def get_labeled_points_from_doc_index(doc_index, doc_classification_map, number_of_terms):
    docs_with_classes = doc_index.map(lambda (doc_id, terms): (doc_id, (terms, doc_classification_map[doc_id])))
    training_vectors = docs_with_classes.map(
        lambda (doc_id, (term_list, classifications)): get_training_vector(classification, term_list,
                                                                           classifications, number_of_terms))
    return training_vectors

get_binary = lambda x: 1 if x > 0 else 0
get_binary = np.vectorize(get_binary)

def get_row_top_N(y_score_row, y_true_row):
    desc_score_indices = np.argsort(y_score_row)[::-1]
    # print y_score_row
    # print y_true_row
    true_indices = np.where(y_true_row ==1)[0]
    # print desc_score_indices
    found = 0
    top_N = 0
    for i, score in enumerate(desc_score_indices):
        if score in true_indices:
            found += 1
            if found == len(true_indices):
                top_N = i + 1
    # print top_N
    return top_N


### Input/Output directories

In [8]:
SAMPLE_RATIO = 0.15

In [9]:
#sc = SparkContext("", "Generate Inverted Index Job")
es_server = "deka.cip.ifi.lmu.de"
es_port = "9200"

original_parent_save_location = "hdfs://deka.cip.ifi.lmu.de/extended_pv/"
save_parent_location = original_parent_save_location

root_location = "/big/s/shalaby/"
exports_location = root_location + "exported_data/"


docs_output = save_parent_location + "docs_output"
postings_list_output = save_parent_location + "postings_list_full.json"

accepted_terms_list_output = original_parent_save_location + "accepted_terms_list_{}.pkl"
accepted_terms_with_scores_list_output = original_parent_save_location + "accepted_terms_with_scores_list_{}.pkl"
postings_list_chi_selected_output = original_parent_save_location + "postings_list_{}.json"
term_df_map_output = original_parent_save_location + "term_df_map_output_{}.json"
doc_index_chi_selected_output = original_parent_save_location + "doc_index_for_postings_{}.json"
term_dictionary_output = original_parent_save_location + "term_dictionary_{}.pkl"


postings_list_training_chi_selected_output = save_parent_location + "training_postings_list_{}.json"
postings_list_validation_chi_selected_output = save_parent_location + "validation_postings_list_{}.json"
postings_list_test_chi_selected_output = save_parent_location + "test_postings_list_{}.json"


# data location
preprocessed_docs_location = root_location + "preprocessed_data/" + "extended_pv_docs_only_for_spark"


# Classification objects, unrelated to sample size
classifications_index_output = exports_location + "extended_pv_classifications_index.pkl"
doc_classifications_map_file = exports_location + "extended_pv_doc_classification_map.pkl"
sections_file = exports_location + "sections.pkl"
classes_file = exports_location + "classes.pkl"
subclasses_file = exports_location + "subclasses.pkl"
valid_classes_file = exports_location + "valid_classes.pkl"
valid_subclasses_file = exports_location + "valid_subclasses.pkl"
classifications_output = exports_location + "classifications.pkl"
doc_lengths_map_output = exports_location + "extended_pv_doc_lengths_map.pkl"
training_docs_list_file = exports_location + "extended_pv_training_docs_list_" + str(SAMPLE_RATIO) + ".pkl"
validation_docs_list_file = exports_location + "extended_pv_validation_docs_list_" + str(SAMPLE_RATIO) + ".pkl"
test_docs_list_file = exports_location + "extended_pv_test_docs_list_" + str(SAMPLE_RATIO) + ".pkl"


training_predictions_sections_output = save_parent_location + "training_predictions_sections_list.pkl"
training_labels_sections_list_output = save_parent_location + "training_labels_sections_list.pkl"
valdiation_predictions_sections_output = save_parent_location + "validation_predictions_sections_list.pkl"
validation_labels_sections_list_output = save_parent_location + "validation_labels_sections_list.pkl"


test_postings_list_output = save_parent_location + "test_postings_list_50000.json"
training_errors_output = save_parent_location + "training_errors.json"
model_output = save_parent_location + "models/" + "iter_" + str(SVM_ITERATIONS) + "_reg_" + str(SVM_REG) + "/"

In [5]:
def get_model_name(method, classification, reg=SVM_REG, iterations=SVM_ITERATIONS):
    return save_parent_location + "models/" + "iter_" + str(iterations) + "_reg_" + str(reg) + "/" + method + "_" + classification + "_model.svm"
def get_data_output_name(method, no_of_features=TOP_N_FEATURES, data_type="training"):
    return save_parent_location + "models/" + data_type + "_data/" + method  + "_data.json"
def get_save_location(location, sample=False):
    if sample:
        return location.replace(save_parent_location, sample_save_parent_location)
    return location

#### Load Classification Objects

In [10]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
sections = pickle.load(open(sections_file))
classes = pickle.load(open(classes_file))
subclasses = pickle.load(open(subclasses_file))
valid_classes = pickle.load(open(valid_classes_file))
valid_subclasses = pickle.load(open(valid_subclasses_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))
test_docs_list = pickle.load(open(test_docs_list_file))
classifications_index = pickle.load(open(classifications_index_output))

CPU times: user 10.2 s, sys: 8 ms, total: 10.2 s
Wall time: 10.6 s


In [12]:
extended_pv_docs = set(training_docs_list) | set(validation_docs_list) | set(test_docs_list)
doc_count = len(extended_pv_docs)

In [13]:
doc_count

395509

## Generating Sparse data format

In [14]:
def create_written_doc_index(term_index, name, data_type="training"):
    doc_index = create_doc_index(term_index, term_dictionary)
    output_name = get_data_output_name(name, data_type=data_type)
    doc_index.map(lambda postings: json.dumps(postings)).repartition(100).saveAsTextFile(output_name)
    doc_index = sc.textFile(output_name).map(get_json_convert_num)# .cache()
    return doc_index

def read_written_doc_index(name, data_type="training"):
    output_name = get_data_output_name(name, data_type=data_type)
    doc_index = sc.textFile(output_name).map(get_json_convert_num)
    return doc_index

Read Training Set

In [22]:
data_type = "test"

In [23]:
tf_doc_index_training = read_written_doc_index("tf", data_type)
sublinear_tf_doc_index_training = read_written_doc_index("tf-sublinear", data_type)
tf_idf_doc_index_training = read_written_doc_index("tf-idf", data_type)
sublinear_tf_idf_doc_index_training = read_written_doc_index("sublinear-tf-idf", data_type)
bm25_doc_index_training = read_written_doc_index("bm25", data_type)

In [ ]:
%%time
ll = [
    ("bm25", bm25_doc_index_training), 
    ("sublinear_tf", sublinear_tf_doc_index_training), 
    ("tf_idf", tf_idf_doc_index_training), 
    ("tf", tf_doc_index_training),
    ("sublinear_tf_idf", sublinear_tf_idf_doc_index_training)
]
for (name, doc_ind) in ll:
    v = sklearn.feature_extraction.DictVectorizer(sparse=True, dtype=float)
    %time yy = doc_ind.collect()
    list_dicts = [y[1] for y in yy]
    doc_ids = [y[0] for y in yy]
    %time X = v.fit_transform(list_dicts)
    %time pickle.dump(X, open("/big/s/shalaby/exported_data/extened_pv_benchmarking_data/{}_{}_sparse_data.pkl".format(name, data_type),"w"))
    pickle.dump(doc_ids, open("/big/s/shalaby/exported_data/extened_pv_benchmarking_data/{}_{}_sparse_docids.pkl".format(name, data_type),"w"))
    del yy
    del X
    print name

CPU times: user 6.12 s, sys: 764 ms, total: 6.88 s
Wall time: 15.8 s
CPU times: user 42.6 s, sys: 4.08 s, total: 46.7 s
Wall time: 46 s
